# Dedupe openfda versus medicare part D

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""
This code demonstrates how to use RecordLink with two comma separated
values (CSV) files. We have listings of products from two different
online stores. The task is to link products between the datasets.

The output will be a CSV with our linkded results.

"""
import os
import csv
import re
import logging
import optparse
import pandas as pd

import dedupe
from unidecode import unidecode

In [2]:
sunshine_manuf = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/pharmCo_info.csv')
df = sunshine_manuf[['Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name']]
df.columns = ['manufacturer_name']
df.to_csv('../Data/Outputs_Cleanup/Sunshine/pharmCo_manufacturer_dedupe_input.csv', index=False)
#df.head()

In [3]:
openfda_manuf = pd.read_csv('../Data/Outputs_Cleanup/FDA/openfda_manufacturer_deduplicated_single_manuf.csv')#df = openfda_manuf[['manufacturer_name']]
df = openfda_manuf[['manufacturer_name']]
#df.columns = ['manufacturer_name']
df.to_csv('../Data/Outputs_Cleanup/FDA/openfda_manufacturer_dedupe_input.csv', index=False)

In [4]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """

    column = unidecode(column)
    column = re.sub('\n', ' ', column)
    column = re.sub('-', '', column)
    column = re.sub('/', ' ', column)
    column = re.sub("'", '', column)
    column = re.sub(",", '', column)
    column = re.sub(":", ' ', column)
    column = re.sub('  +', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column:
        column = None
    return column


def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID.
    """

    data_d = {}

    with open(filename) as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            clean_row = dict([(k, preProcess(v)) for (k, v) in row.items()])
            data_d[filename + str(i)] = dict(clean_row)

    return data_d

In [5]:
 # ## Setup
output_file = '../Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_output.csv'
settings_file = '../Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_learned_settings'
training_file = '../Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_training.json'
    
retrain = input('Do you want to add on to your training (y/n). If you wanted to start over, delete your .json file')
isretrain = True if retrain == 'y' else False

if isretrain == True:
    try:
        os.remove(settings_file)
    except:
        print('Your settings file appears to not have existed.')

left_file = '../Data/Outputs_Cleanup/Sunshine/pharmCo_manufacturer_dedupe_input.csv'
right_file = '../Data/Outputs_Cleanup/FDA/openfda_manufacturer_dedupe_input.csv'

print('importing data ...')
data_1 = readData(left_file)
data_2 = readData(right_file)

def descriptions():
    for dataset in (data_1, data_2):
        for record in dataset.values():
            yield record['description']

# ## Training

if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as sf:
        linker = dedupe.StaticRecordLink(sf)

else:
    # Define the fields the linker will pay attention to
    #
    # Notice how we are telling the linker to use a custom field comparator
    # for the 'price' field.
    fields = [
        {'field': 'manufacturer_name', 'type': 'String'},
        #{'field': 'title', 'type': 'Text', 'corpus': descriptions()},
        #{'field': 'description', 'type': 'Text',
        # 'has missing': True, 'corpus': descriptions()},
        #{'field': 'price', 'type': 'Price', 'has missing': True}
    ]

    # Create a new linker object and pass our data model to it.
    linker = dedupe.RecordLink(fields)

    # If we have training data saved from a previous run of linker,
    # look for it an load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file) as tf:
            linker.prepare_training(data_1,
                                    data_2,
                                    training_file=tf,
                                    sample_size=15000)
    else:
        linker.prepare_training(data_1, data_2, sample_size=15000)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as matches
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.console_label(linker)

    linker.train()

    # When finished, save our training away to disk
    with open(training_file, 'w') as tf:
        linker.write_training(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf:
        linker.write_settings(sf)

# ## Blocking

# ## Clustering

# Find the threshold that will maximize a weighted average of our
# precision and recall.  When we set the recall weight to 2, we are
# saying we care twice as much about recall as we do precision.
#
# If we had more data, we would not pass in all the blocked data into
# this function but a representative sample.

print('clustering...')
linked_records = linker.join(data_1, data_2, 0.0)

print('# duplicate sets', len(linked_records))
# ## Writing Results

# Write our original data back out to a CSV with a new column called
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
for cluster_id, (cluster, score) in enumerate(linked_records):
    for record_id in cluster:
        cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                         'Link Score': score}

with open(output_file, 'w') as f:

    header_unwritten = True

    for fileno, filename in enumerate((left_file, right_file)):
        with open(filename) as f_input:
            reader = csv.DictReader(f_input)

            if header_unwritten:

                fieldnames = (['Cluster ID', 'Link Score', 'source file'] +
                              reader.fieldnames)

                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()

                header_unwritten = False

            for row_id, row in enumerate(reader):

                record_id = filename + str(row_id)
                cluster_details = cluster_membership.get(record_id, {})
                row['source file'] = fileno
                row.update(cluster_details)

                writer.writerow(row)

Do you want to add on to your training (y/n). If you wanted to start over, delete your .json file y


INFO:dedupe.api:reading training from file


importing data ...
reading labeled examples from  ../Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_training.json


INFO:dedupe.canopy_index:Removing stop word  c
INFO:dedupe.canopy_index:Removing stop word ac
INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word ca
INFO:dedupe.canopy_index:Removing stop word co
INFO:dedupe.canopy_index:Removing stop word ha
INFO:dedupe.canopy_index:Removing stop word in
INFO:dedupe.canopy_index:Removing stop word l 
INFO:dedupe.canopy_index:Removing stop word ma
INFO:dedupe.canopy_index:Removing stop word on
INFO:dedupe.canopy_index:Removing stop word ra
INFO:dedupe.canopy_index:Removing stop word rm
INFO:dedupe.canopy_index:Removing stop word ti
INFO:dedupe.canopy_index:Removing stop word  l
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ch
INFO:dedupe.canopy_index:Removing stop word en
INFO:dedupe.canopy_index:Removing stop word lt
INFO:dedupe.canopy_index:Removing stop word n 
INFO:dedupe.canopy_index:Removing stop word nt
INFO:dedupe.canopy_index:Removing stop word o 
INFO:dedupe.c

starting active labeling...


...

 f


Finished labeling
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.1873522082478936
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameSevenCharStartPredicate, manufacturer_name), TfidfNGramSearchPredicate: (0.2, manufacturer_name))
INFO:dedupe.training:(SimplePredicate: (sameFiveCharStartPredicate, manufacturer_name), SimplePredicate: (commonTwoTokens, manufacturer_name))


clustering...


INFO:dedupe.canopy_index:Removing stop word  c
INFO:dedupe.canopy_index:Removing stop word ac
INFO:dedupe.canopy_index:Removing stop word ar
INFO:dedupe.canopy_index:Removing stop word ca
INFO:dedupe.canopy_index:Removing stop word co
INFO:dedupe.canopy_index:Removing stop word ha
INFO:dedupe.canopy_index:Removing stop word in
INFO:dedupe.canopy_index:Removing stop word l 
INFO:dedupe.canopy_index:Removing stop word ma
INFO:dedupe.canopy_index:Removing stop word on
INFO:dedupe.canopy_index:Removing stop word ra
INFO:dedupe.canopy_index:Removing stop word rm
INFO:dedupe.canopy_index:Removing stop word ti
INFO:dedupe.canopy_index:Removing stop word  l
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word ch
INFO:dedupe.canopy_index:Removing stop word en
INFO:dedupe.canopy_index:Removing stop word lt
INFO:dedupe.canopy_index:Removing stop word n 
INFO:dedupe.canopy_index:Removing stop word nt
INFO:dedupe.canopy_index:Removing stop word o 
INFO:dedupe.c

# duplicate sets 389


In [ ]:
df = pd.read_csv('../Data/Outputs_Cleanup/Manufacturer_entity_matching/data_matching_output.csv')
#df.sort_values('Cluster ID')
df_fda = df[df['source file'] == 0]
df_sunshine = df[df['source file'] == 1]
df_fda = df_fda.merge(fda_drugs[['manufacturer_name', 'manuf__id']], how='left', on='brand_name')
df_partD = df_partD.merge(medD_drugs[['brand_name', 'MedD_drug_Id']], how='left', on='brand_name')

drug_merge = pd.concat([df_fda, df_partD], axis=0).sort_values('Cluster ID')
drug_merge['fda_drug_id'] = drug_merge['fda_drug_id'].fillna('[]')
drug_merge['MedD_drug_Id'] = drug_merge['MedD_drug_Id'].fillna('[]')
drug_merge = drug_merge.groupby('Cluster ID').agg(list)

def pick_list(x):
    my_pick = None
    for each in x:
        if each == '[]':
            continue
        else:
            my_pick = each
    return my_pick

drug_merge['fda_drug_id'] = drug_merge['fda_drug_id'].apply(lambda x: pick_list(x))
drug_merge['MedD_drug_Id'] = drug_merge['MedD_drug_Id'].apply(lambda x: pick_list(x))

def pick_longest(x):
    longest = None
    longest_len = 0
    for idx, each in enumerate(x):
        if idx == 0:
            longest = each
            longest_len = len(each)
            continue
        if len(each) > longest_len:
            longest = each
            longest_len = len(each)  
    return longest

drug_merge['brand_name'] = drug_merge['brand_name'].apply(lambda x: pick_longest(x))
drug_merge = drug_merge[['brand_name', 'fda_drug_id', 'MedD_drug_Id']]
drug_merge.to_csv('../../Data/Outputs_Cleanup/FDA_partD_drug_matching/fda_partD_drugs_matched.csv', index=False)